In [11]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [17]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tqdm as tqdm

In [13]:
#ACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
#PRUEBO CON MENOS DATOS
#ACA SUBO LOS ARCHIVOS DE TFRECORD
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\tfrecord_data' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 5  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

In [9]:
'''#SEPARO EN X E Y
X = []
y = []

#itero los lotes
for batch_data in batched_dataset:
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    #cargo los scaleograms desde los archivos .npy
    batch_X = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_X.append(spectrogram)
    
#//////NUEVO

    #primero tengo que convertirlo en un archivo numpy porque sino no quiere
    batch_X = np.array(batch_X)
    
    X_shape = batch_X.shape
    shape_0 = X_shape[0]

    # Aplanar
    batch_X = batch_X.reshape(shape_0, -1)
    
    #NORMALIZO X en rango [-1, 1] para que el cero quede en el centro
    min_val = np.min(batch_X)
    max_val = np.max(batch_X)

    X = -1 + 2 * (batch_X - min_val) / (max_val - min_val)
    
    variance_to_keep = 0.95  #me quedo con el 95% de la varianza, esto es flexible (mientras más porcentaje, mayor va  quedar la dimensionalidad de mi X)

    pca = PCA()
    batch_X = pca.fit_transform(batch_X)  #aplico pca para los datos escalados

    #Encuentro el número mínimo de componentes principales para alcanzar la varianza deseada
    cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
    #('n_components_to_keep' contendrá el número mínimo de componentes principales necesarios para explicar al menos el 95% de la varianza en los datos.)
    n_components_to_keep = np.argmax(cumulative_variance_ratio >= variance_to_keep) + 1 #chona me dijo que me deberían quedar por ejemplo 1000 atributos

    #Selecciono la cant de componentes principales que me salió
    batch_X = batch_X[:, :n_components_to_keep]
    
#//////NUEVO
    
    #agrego los scaleograms y labels a las listas X y y
    X.concatenate(batch_X)
    y.extend(labels)


#convierto X e y a formato .npy
X = np.concatenate(X, axis=0)
y = np.array(y)

print('Ya se separó todo!')
print('Salió todo bien, tranquina :))')'''

AttributeError: 'numpy.ndarray' object has no attribute 'concatenate'

In [20]:
from tqdm import tqdm

# Define el número máximo de componentes principales que deseas mantener para todos los lotes
max_components = 1000  # Puedes ajustar este valor según tus necesidades

X_list = []  # Lista temporal para X
y_list = []  # Lista temporal para y

# Crea un objeto tqdm para rastrear el progreso
pbar = tqdm(total=len(filenames))

# Itera los lotes
for batch_data in batched_dataset:
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    # Cargar los scaleograms desde los archivos .npy
    batch_X = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_X.append(spectrogram)
    
    # Convierte batch_X en un arreglo NumPy
    batch_X = np.array(batch_X)
    
    batch_X = batch_X.reshape((batch_X.shape[0], -1))  # Aplanar los datos
    
    # NORMALIZA X en rango [-1, 1] para que el cero quede en el centro
    min_val = np.min(batch_X)
    max_val = np.max(batch_X)

    batch_X = -1 + 2 * (batch_X - min_val) / (max_val - min_val)
    
    variance_to_keep = 0.95  # Me quedo con el 95% de la varianza

    pca = PCA()
    batch_X = pca.fit_transform(batch_X)  # Aplica PCA para los datos escalados

    # Asegúrate de que todos los lotes tengan la misma cantidad de componentes principales
    n_components_to_keep = min(max_components, batch_X.shape[1])
    if batch_X.shape[1] < max_components:
        # Rellena con ceros las columnas faltantes si es necesario
        zeros_to_add = max_components - batch_X.shape[1]
        zeros = np.zeros((batch_X.shape[0], zeros_to_add))
        batch_X = np.hstack((batch_X, zeros))
    else:
        batch_X = batch_X[:, :n_components_to_keep]

    # Agrega los scaleograms y labels a X_list e y_list
    X_list.append(batch_X)
    y_list.extend(labels)

    # Actualiza la barra de progreso
    pbar.update(len(paths))  # Actualiza la barra de progreso según la cantidad de archivos en el lote

# Convierte X_list e y_list en arreglos NumPy
X = np.concatenate(X_list, axis=0)
y = np.array(y_list)

# Cierra la barra de progreso
pbar.close()

29588it [18:17, 26.96it/s]                                                                                             


In [ ]:
'''X_shape = X.shape
shape_0 = X_shape[0]
X_shape'''

In [13]:
# Aplanar tus datos
X = X.reshape(shape_0, -1)

In [14]:
#NORMALIZO X en rango [-1, 1] para que el cero quede en el centro
min_val = np.min(X)
max_val = np.max(X)

X = -1 + 2 * (X - min_val) / (max_val - min_val)

#X = (X - X.min()) / (X.max() - X.min()) [0, 1]

In [1]:
X.shape

NameError: name 'X' is not defined

In [ ]:
from sklearn.decomposition import PCA

variance_to_keep = 0.95  #me quedo con el 95% de la varianza, esto es flexible (mientras más porcentaje, mayor va  quedar la dimensionalidad de mi X)

pca = PCA()
X = pca.fit_transform(X)  #aplico pca para los datos escalados

#Encuentro el número mínimo de componentes principales para alcanzar la varianza deseada
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
#('n_components_to_keep' contendrá el número mínimo de componentes principales necesarios para explicar al menos el 95% de la varianza en los datos.)
n_components_to_keep = np.argmax(cumulative_variance_ratio >= variance_to_keep) + 1 #chona me dijo que me deberían quedar por ejemplo 1000 atributos

#Selecciono la cant de componentes principales que me salió
X = X[:, :n_components_to_keep]

In [ ]:
print("Me quedé con " + str(n_components_to_keep) + " componentes principales")
X.shape

In [ ]:
print("Varianza explicada por cada componente principal:", pca.explained_variance_ratio_)

In [21]:
#LOS GUARDO EN MI COMPUTADORA
#np.save('X.npy', X) está guardada normalizada en rango [0, 1], habría que cambiar eso
np.save('X_PCA.npy', X) #esto es para probar
np.save('y_PCA.npy', y)

In [ ]:
#no se si lo que hice está bien, chona me dijo que usara PaCMAP pero no lo entiendo, y acá encontre que con sklearn medio que se puede hacer asi que :)

In [ ]:
#hago una función?